Copy the file **spy_dte_0_iron_condor.csv.gz** located in https://github.com/crapher/medium/tree/main/11.OptionsBacktest/data to your colab folder

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
# Constants
DEBUG = 1
OPTIONS_FILE='./spy_dte_0_iron_condor.csv.gz'

FEES_PER_CONTRACT = 0.6
CONTRACTS = 10

HOUR_OPEN = 11
MINUTE_OPEN = 45
STRIKES = 1

In [3]:
### Helper functions ###
def get_iron_condor_strikes(df, strikes):

    put_sell  = int(df['close_underlying'].values[0]) - strikes
    call_sell = int(df['close_underlying'].values[0]) + strikes
    put_buy   = put_sell - 1
    call_buy  = call_sell + 1

    return (put_buy, put_sell, call_sell, call_buy)

def get_iron_condor_cost(df, put_buy, put_sell, call_sell, call_buy):

    # Open Iron Condor
    df = df.sort_values('date', ascending=True)

    stock_price     = df[(df['strike'] == put_buy)   & (df['kind'] == 'P')]['close_underlying'].values[0]
    put_buy_price   = df[(df['strike'] == put_buy)   & (df['kind'] == 'P')]['close'].values[0]
    put_sell_price  = df[(df['strike'] == put_sell)  & (df['kind'] == 'P')]['close'].values[0]
    call_sell_price = df[(df['strike'] == call_sell) & (df['kind'] == 'C')]['close'].values[0]
    call_buy_price  = df[(df['strike'] == call_buy)  & (df['kind'] == 'C')]['close'].values[0]

    open_cost  = -100 * put_buy_price
    open_cost +=  100 * put_sell_price
    open_cost +=  100 * call_sell_price
    open_cost += -100 * call_buy_price
    open_cost *= CONTRACTS

    if DEBUG:
        tqdm.write(f' OPEN ({stock_price}): {put_buy} @ {put_buy_price} - {put_sell} @ {put_sell_price} - {call_sell} @ {call_sell_price} - {call_buy} @ {call_buy_price}')

    # Close Iron Condor
    stock_price     = df[(df['strike'] == put_buy)   & (df['kind'] == 'P')]['close_underlying'].values[-1]
    put_buy_price   = df[(df['strike'] == put_buy)   & (df['kind'] == 'P')]['close'].values[-1]
    put_sell_price  = df[(df['strike'] == put_sell)  & (df['kind'] == 'P')]['close'].values[-1]
    call_sell_price = df[(df['strike'] == call_sell) & (df['kind'] == 'C')]['close'].values[-1]
    call_buy_price  = df[(df['strike'] == call_buy)  & (df['kind'] == 'C')]['close'].values[-1]

    if DEBUG:
        tqdm.write(f'CLOSE ({stock_price}): {put_buy} @ {put_buy_price} - {put_sell} @ {put_sell_price} - {call_sell} @ {call_sell_price} - {call_buy} @ {call_buy_price}')

    close_cost  =  100 * put_buy_price
    close_cost += -100 * put_sell_price
    close_cost += -100 * call_sell_price
    close_cost +=  100 * call_buy_price
    close_cost *= CONTRACTS

    if DEBUG:
        tqdm.write(f'**************** Processed Date: {df["date"].dt.date.values[0]} - Result: {open_cost + close_cost:7.2f} ****************')

    return open_cost + close_cost

In [4]:
### Read File ###
df = pd.read_csv(OPTIONS_FILE, header=0)
df['date'] = pd.to_datetime(df['date'])

In [5]:
### Get all the expiration dates ###
expirations = df['expire_date'].unique().tolist()

In [6]:
### Get P&L by day ###
pnl = []

df_cost = df[(df.date.dt.hour > HOUR_OPEN) | ((df.date.dt.hour == HOUR_OPEN) & (df.date.dt.minute >= MINUTE_OPEN))]
df_open = df_cost[(df_cost.date.dt.hour == HOUR_OPEN) & (df_cost.date.dt.minute == MINUTE_OPEN)]

for expiration in tqdm(expirations):

    try:
        (put_buy, put_sell, call_sell, call_buy) = get_iron_condor_strikes(df_open[df_open['expire_date'] == expiration], STRIKES)
        cost = get_iron_condor_cost(df_cost[df_cost['expire_date'] == expiration], put_buy, put_sell, call_sell, call_buy)

        pnl.append([expiration, cost])
    except Exception as ex:
        continue

  1%|          | 2/370 [00:00<00:30, 11.88it/s]

 OPEN (420.2311): 418 @ 0.13 - 419 @ 0.25 - 421 @ 0.27 - 422 @ 0.07
CLOSE (420.84): 418 @ 0.01 - 419 @ 0.02 - 421 @ 0.05 - 422 @ 0.01
**************** Processed Date: 2021-06-21 - Result:  270.00 ****************
 OPEN (423.22): 421 @ 0.07 - 422 @ 0.18 - 424 @ 0.21 - 425 @ 0.05
CLOSE (422.6): 421 @ 0.01 - 422 @ 0.04 - 424 @ 0.01 - 425 @ 0.05
**************** Processed Date: 2021-06-23 - Result:  280.00 ****************
 OPEN (426.25): 424 @ 0.06 - 425 @ 0.11 - 427 @ 0.15 - 428 @ 0.03
CLOSE (426.56): 424 @ 0.01 - 425 @ 0.01 - 427 @ 0.04 - 428 @ 0.01
**************** Processed Date: 2021-06-25 - Result:  140.00 ****************


  2%|▏         | 6/370 [00:00<00:29, 12.53it/s]

 OPEN (426.415): 424 @ 0.03 - 425 @ 0.09 - 427 @ 0.19 - 428 @ 0.04
CLOSE (427.48): 424 @ 0.01 - 425 @ 0.01 - 427 @ 0.51 - 428 @ 0.01
**************** Processed Date: 2021-06-28 - Result: -290.00 ****************
 OPEN (427.97): 425 @ 0.04 - 426 @ 0.07 - 428 @ 0.42 - 429 @ 0.1
CLOSE (428.07): 425 @ 0.01 - 426 @ 0.02 - 428 @ 0.25 - 429 @ 0.01
**************** Processed Date: 2021-06-30 - Result:  100.00 ****************
 OPEN (432.19): 430 @ 0.04 - 431 @ 0.08 - 433 @ 0.07 - 434 @ 0.01
CLOSE (433.635): 430 @ 0.01 - 431 @ 0.01 - 433 @ 0.73 - 434 @ 0.05
**************** Processed Date: 2021-07-02 - Result: -580.00 ****************


  2%|▏         | 8/370 [00:00<00:29, 12.39it/s]

...

 OPEN (427.49): 425 @ 0.08 - 426 @ 0.18 - 428 @ 0.42 - 429 @ 0.17
CLOSE (426.6): 425 @ 0.01 - 426 @ 0.01 - 428 @ 0.01 - 429 @ 0.01
**************** Processed Date: 2023-06-07 - Result:  350.00 ****************
 OPEN (428.1401): 426 @ 0.11 - 427 @ 0.24 - 429 @ 0.27 - 430 @ 0.09
CLOSE (429.125): 426 @ 0.01 - 427 @ 0.01 - 429 @ 0.2 - 430 @ 0.01
**************** Processed Date: 2023-06-08 - Result:  120.00 ****************
 OPEN (430.07): 428 @ 0.16 - 429 @ 0.32 - 431 @ 0.36 - 432 @ 0.14
CLOSE (429.89): 428 @ 0.01 - 429 @ 0.01 - 431 @ 0.01 - 432 @ 0.01
**************** Processed Date: 2023-06-09 - Result:  380.00 ****************


 99%|█████████▉| 368/370 [00:32<00:00, 11.50it/s]

 OPEN (431.145): 429 @ 0.1 - 430 @ 0.22 - 432 @ 0.23 - 433 @ 0.07
CLOSE (433.79): 429 @ 0.01 - 430 @ 0.01 - 432 @ 1.72 - 433 @ 0.8
**************** Processed Date: 2023-06-12 - Result: -640.00 ****************
 OPEN (436.31): 434 @ 0.13 - 435 @ 0.26 - 437 @ 0.45 - 438 @ 0.21
CLOSE (436.66): 434 @ 0.01 - 435 @ 0.01 - 437 @ 0.02 - 438 @ 0.01
**************** Processed Date: 2023-06-13 - Result:  360.00 ****************
 OPEN (438.5): 436 @ 0.66 - 437 @ 0.95 - 439 @ 1.34 - 440 @ 0.93
CLOSE (437.18): 436 @ 0.02 - 437 @ 0.17 - 439 @ 0.01 - 440 @ 0.01
**************** Processed Date: 2023-06-14 - Result:  550.00 ****************


100%|██████████| 370/370 [00:33<00:00, 11.20it/s]

 OPEN (440.9): 438 @ 0.15 - 439 @ 0.25 - 441 @ 0.76 - 442 @ 0.38
CLOSE (442.6): 438 @ 0.01 - 439 @ 0.01 - 441 @ 1.51 - 442 @ 0.65
**************** Processed Date: 2023-06-15 - Result: -380.00 ****************
 OPEN (441.05): 439 @ 0.2 - 440 @ 0.39 - 442 @ 0.42 - 443 @ 0.18
CLOSE (439.43): 439 @ 0.03 - 440 @ 0.61 - 442 @ 0.01 - 443 @ 0.01
**************** Processed Date: 2023-06-16 - Result: -150.00 ****************


In [7]:
### Generate result ###

# Calculate Net Result
df_result = pd.DataFrame(pnl, columns=['expire_date','gross_result'])
df_result['net_result'] = df_result['gross_result'] - CONTRACTS * FEES_PER_CONTRACT * 4 # Let them expire
df_result['wins'] = np.where(df_result['net_result'] > 0, 1, 0)
df_result['losses'] = np.where(df_result['net_result'] < 0, 1, 0)

In [8]:
# Show the parameters
print(f' PARAMETERS '.center(70, '*'))
print(f'* Hour Open: {HOUR_OPEN}')
print(f'* Minute Open: {MINUTE_OPEN}')
print(f'* Strikes From Price: {STRIKES}')

# Show the Total Result
print(f' SUMMARIZED RESULT '.center(70, '*'))
print(f'* Trading Days: {len(df_result)}')
print(f'* Gross PnL: $ {df_result["gross_result"].sum():.2f}')
print(f'* Net PnL: $ {df_result["net_result"].sum():.2f}')
print(f'* Win Rate: {100 * (df_result["wins"].sum() / (df_result["wins"].sum() + df_result["losses"].sum())):.2f} %')

# Show The Monthly Result
print(f' MONTHLY DETAIL RESULT '.center(70, '*'))
df_monthly = df_result[['expire_date','gross_result','net_result','wins','losses']]
df_monthly['year_month'] = df_monthly['expire_date'].str[0:7]
df_monthly = df_monthly.drop('expire_date', axis=1)
df_monthly = df_monthly.groupby(['year_month']).sum()

print(df_monthly)

***************************** PARAMETERS *****************************
* Hour Open: 11
* Minute Open: 45
* Strikes From Price: 1
************************* SUMMARIZED RESULT **************************
* Trading Days: 360
* Gross PnL: $ 9970.00
* Net PnL: $ 1330.00
* Win Rate: 52.22 %
*********************** MONTHLY DETAIL RESULT ************************
            gross_result  net_result  wins  losses
year_month                                        
2021-06            500.0       380.0     4       1
2021-07            690.0       378.0     8       5
2021-08            200.0      -112.0     9       4
2021-09           1410.0      1074.0    10       4
2021-10          -2630.0     -2942.0     4       9
2021-11            300.0       -12.0     8       5
2021-12            -30.0      -270.0     4       6
2022-01           1440.0      1224.0     5       4
2022-02           1660.0      1396.0     7       4
2022-03          -2050.0     -2386.0     6       8
2022-04          -2220.0     -253